In [3]:
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans

In [4]:
spark = SparkSession.builder.getOrCreate()

In [5]:
data_path = '/Users/quangly/github/Spark SQL/Data'

In [6]:
# exploratory data analysis, Clustering. natural groupings
#divide data set into 3 logical datasets 

In [7]:
json_df2_path = data_path + "/utilization.json"
df_util = spark.read.format("json").load(json_df2_path)

In [8]:
df_util.show()

+---------------+-------------------+-----------+---------+-------------+
|cpu_utilization|     event_datetime|free_memory|server_id|session_count|
+---------------+-------------------+-----------+---------+-------------+
|           0.57|03/05/2019 08:06:14|       0.51|      100|           47|
|           0.47|03/05/2019 08:11:14|       0.62|      100|           43|
|           0.56|03/05/2019 08:16:14|       0.57|      100|           62|
|           0.57|03/05/2019 08:21:14|       0.56|      100|           50|
|           0.35|03/05/2019 08:26:14|       0.46|      100|           43|
|           0.41|03/05/2019 08:31:14|       0.58|      100|           48|
|           0.57|03/05/2019 08:36:14|       0.35|      100|           58|
|           0.41|03/05/2019 08:41:14|        0.4|      100|           58|
|           0.53|03/05/2019 08:46:14|       0.35|      100|           62|
|           0.51|03/05/2019 08:51:14|        0.6|      100|           45|
|           0.32|03/05/2019 08:56:14| 

In [9]:
# group based on cpu, free memory, session count
# ml libraries work with vectors (like an array)
# create vector to store all 3 values called "features"

In [12]:
vectorAssembler = VectorAssembler(inputCols=["cpu_utilization", "free_memory", "session_count"], outputCol="features")

In [13]:
# vectorized cluster data frame.

In [14]:
vcluster_df = vectorAssembler.transform(df_util)

In [15]:
vcluster_df.show()

+---------------+-------------------+-----------+---------+-------------+----------------+
|cpu_utilization|     event_datetime|free_memory|server_id|session_count|        features|
+---------------+-------------------+-----------+---------+-------------+----------------+
|           0.57|03/05/2019 08:06:14|       0.51|      100|           47|[0.57,0.51,47.0]|
|           0.47|03/05/2019 08:11:14|       0.62|      100|           43|[0.47,0.62,43.0]|
|           0.56|03/05/2019 08:16:14|       0.57|      100|           62|[0.56,0.57,62.0]|
|           0.57|03/05/2019 08:21:14|       0.56|      100|           50|[0.57,0.56,50.0]|
|           0.35|03/05/2019 08:26:14|       0.46|      100|           43|[0.35,0.46,43.0]|
|           0.41|03/05/2019 08:31:14|       0.58|      100|           48|[0.41,0.58,48.0]|
|           0.57|03/05/2019 08:36:14|       0.35|      100|           58|[0.57,0.35,58.0]|
|           0.41|03/05/2019 08:41:14|        0.4|      100|           58| [0.41,0.4,58.0]|

In [16]:
# KMeans ML algorithm
# K = number of clusters
# takes a seed
# kmeans a structure to run kmeans algorithm

In [17]:
kmeans = KMeans().setK(3)
kmeans = kmeans.setSeed(1)

In [ ]:
# fit - take input data and apply algorithm

In [19]:
kmodel = kmeans.fit(vcluster_df)

In [ ]:
# Critical thing of KMeans model is Cluster Center
# we have set of 3 centers, each center is represented by 3 values similar to x, y z in space \ 
# but cpu utilization, free memory, and sessioncount.
# each of these values indicate the center of these 3 values.
# all of our rows in our dataframe fit or fall into one of these 3 clusters.
# we can measure by the distance of the feature vector of each of the center

In [20]:
kmodel.clusterCenters()

[array([ 0.52047775,  0.47836303, 51.79927162]),
 array([ 0.62881549,  0.37094643, 70.43030159]),
 array([ 0.71931575,  0.28104316, 88.23965784])]